# 3.0 Linked Lists

Add some common functions for representing linked lists and converting between linked lists and python lists.

In [1]:
class ListNode(object):
    """Node in a singly linked list."""
    
    def __init__(self, data, next=None):
        self.data = data
        self.next = next

        
def make_linked_list(elems):
    """Return a linked list initialized from python list."""
    head, node = None, None
    for elem in elems:
        if head is None:
            head = ListNode(elem)
            node = head
        else:
            node.next = ListNode(elem)
            node = node.next
    return head


def make_list(head):
    """Return a python list initialized from linked list."""
    elems, node = [], head
    while node is not None:
        elems.append(node.data)
        node = node.next
    return elems

## 3.1 Reverse a linked list

### Problem Statement
Reverse the nodes of a linked list without using any additional storage.

In [2]:
import collections
import unittest


def reverse_list(head):
    """Reverse the singly linked list in-place."""
    node = head
    while node.next is not None:
        tmp = node.next
        node.next = node.next.next  # Bypass current next.
        tmp.next = head  # Make next the new head.
        head = tmp
    return head


class ReverseListTest(unittest.TestCase):

    def test_reverse_list(self):
        case = collections.namedtuple('case', ['input','expected'])
        cases = [
            case([1,2,3,4], [4,3,2,1]),
            case([1], [1]),
        ]
        for c in cases:
            in_list = make_linked_list(c.input)
            rev_list = reverse_list(in_list)
            self.assertEqual(make_list(rev_list), c.expected)


unittest.main(ReverseListTest(), argv=[''], verbosity=2, exit=False)

test_reverse_list (__main__.ReverseListTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.004s

OK


## 3.2 Implement grade school addition using lists

### Problem Statement
Implement grade school addition for two numbers represented as lists of digits from `0-9` with the **least** significant digit appearing first.

In [3]:
import collections
import unittest


def sum_digits(a, b):
    """Compute sum of a and b represented as digits in linked list."""
    head, node, nodea, nodeb, carry = None, None, a, b, 0
    while nodea is not None or nodeb is not None:
        # Add the digits.
        digita = 0 if nodea is None else nodea.data
        digitb = 0 if nodeb is None else nodeb.data
        sumdigits = digita + digitb + carry
        if sumdigits > 9:
            carry = sumdigits // 10  # Round toward zero.
            sumdigits = sumdigits % 10
        # Append the digits to the sum.
        if head is None:
            head = ListNode(sumdigits)
            node = head
        else:
            node.next = ListNode(sumdigits)
            node = node.next
        # Advance the pointers.
        nodea = None if nodea is None else nodea.next
        nodeb = None if nodeb is None else nodeb.next
        
    # Handle any remaining carry digits.
    if carry > 0:
        node.next = ListNode(carry)

    return head
    

class SumDigitsTest(unittest.TestCase):

    def test_sum_digits(self):
        case = collections.namedtuple('case', ['a','b','expected'])
        cases = [
            # One-digit, no carry.
            case([0],[0], [0]),
            case([1],[2], [3]),
            # One-digit, carry.
            case([6],[4], [0,1]),
            case([5],[6], [1,1]),
            # Two-digit, no carry.
            case([2,1],[4,3], [6,4]),
            # Two-digit, carry.
            case([1,6],[8,8], [9,4,1]),
            # Mix of unequal number of digits.
            case([3,1],[5], [8,1]),
            case([2,1],[0,2,3], [2,3,3]),
        ]
        for c in cases:
            a, b = make_linked_list(c.a), make_linked_list(c.b)
            sum_ab = sum_digits(a, b)
            self.assertEqual(make_list(sum_ab), c.expected)


unittest.main(SumDigitsTest(), argv=[''], verbosity=2, exit=False)

test_sum_digits (__main__.SumDigitsTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.005s

OK


## 3.3 Rearrange a linked list to alternate low-high

### Problem Statement
Implement a function that given a list of integers in any order, returns the list with values arranged in alternating sequence of low followed by high.

#### Examples
```
input:  [1,2,3,4,5]
output: [1,3,2,5,4]
```

In [4]:
import collections
import unittest


def low_high(head):
    """Rearrange nodes of singly linked list into alternating low-high."""
    assert not(head.next is None), 'invalid: list requires >= 2 nodes'

    # Partition the list using two pointers.
    # p1 and p2 point to adjacent pairs of nodes.
    # p1_low is toggled to maintain the alternating low-high sequence. 
    p1, p2, p1_low = head, head.next, True
    while p2 is not None:
        # Swap data when invariant is violated.
        if p1_low and not(p1.data < p2.data):
            p1.data, p2.data = p2.data, p1.data
        elif not(p1_low) and p1.data < p2.data:
            p1.data, p2.data = p2.data, p1.data
        p1, p2, p1_low = p2, p2.next, not(p1_low)


class LowHighTest(unittest.TestCase):

    def test_low_high(self):
        case = collections.namedtuple('case', ['input','expected'])
        cases = [
            # Sorted ascending order.
            case([1,2,3,4,5], [1,3,2,5,4]),
            # Sorted descending order.
            case([5,4,3,2,1], [4,5,2,3,1]),
            # Ascending order concatenated with descending order.
            case([1,2,3,3,2,1], [1,3,2,3,1,2]),
            # Descending order concatenated with ascending order.
            case([3,2,1,1,2,3,], [2,3,1,2,1,3]),
            # No swaps required.
            case([1,3,2,5,4], [1,3,2,5,4]),
            # Random.
            case([1,4,9,2,7,5,3,8,6], [1,9,2,7,4,5,3,8,6]),
        ]
        for c in cases:
            ll = make_linked_list(c.input)
            low_high(ll)
            self.assertEqual(make_list(ll), c.expected)


unittest.main(LowHighTest(), argv=[''], verbosity=2, exit=False)

test_low_high (__main__.LowHighTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.011s

OK


## 3.4 Overlapping lists without cycles

### Problem Statement
Given two acyclic lists of size $m$ and $n$, determine whether the lists overlap with each other and return the first overlapping node, if any. 

#### Clarification
For this problem, lists overlap when the last $p$ nodes ($p \leq m$ and $p \leq n$) are common between the lists.

In [5]:
import collections
import unittest


def list_length(head):
    """Return the length of a linked list."""
    node, length = head, 0
    while node is not None:
        length += 1
        node = node.next
    return length


def list_overlap(a, b):
    """Return first overlapping node, if any, between linked lists."""
    # Compute the length of each list.
    m, n = list_length(a), list_length(b)
    
    # Advance the longer list by the difference in size, if any.
    nodea, nodeb, diff = a, b, abs(m-n)
    if m > n:
        while diff > 0:
            nodea = nodea.next
            diff -= 1
    elif m < n:
        while diff > 0:
            nodeb = nodeb.next
            diff -= 1
    
    # Advance the nodes in tandem until overlapping nodes are found.
    while not(nodea is None) and not(nodeb is None):
        if nodea == nodeb:
            break
        nodea, nodeb = nodea.next, nodeb.next
    
    return nodea  # Returns None if no overlap found.


class ListOverlapTest(unittest.TestCase):
    
    def setUp(self):
        self.listA = make_linked_list([1,2,3])
        self.listB = make_linked_list([4,5,6,7,8])
        self.listC = make_linked_list([9,10,11,12])
        # Create a list C and append A.
        self.listCUA = make_linked_list(make_list(self.listC))
        node = self.listCUA
        while node.next is not None:
            node = node.next
        node.next = self.listA

    def test_list_overlap(self):
        case = collections.namedtuple('case', ['a','b','expected'])
        cases = [
            # No overlap.
            case(self.listA, self.listB, None),
            case(self.listB, self.listA, None),
            case(self.listA, self.listC, None),
            case(self.listB, self.listC, None),
            # Overlap.
            case(self.listA, self.listCUA, [1,2,3]),
            case(self.listCUA, self.listA, [1,2,3]),
        ]
        for c in cases:
            rcv = list_overlap(c.a, c.b)
            rcv = rcv if rcv is None else make_list(rcv)
            self.assertEqual(rcv, c.expected)


unittest.main(ListOverlapTest(), argv=[''], verbosity=2, exit=False)

test_list_overlap (__main__.ListOverlapTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.003s

OK
